In [1]:
from urllib.request import urlopen
import re
import pandas as pd
import numpy as np
import sqlite3

In [2]:
def divide(string):
    if 'https' in string:
        link = re.findall('<a href="([a-z\./:-]*)">', string)[0]
        # string = string.replace(f'<a href="{string}">', '')
        name = re.findall('>([A-ZÁÉÍÓÚÑa-záéíóúñ ]*)<', string)[0].strip()
        return [name, link]
    else:
        name = re.findall('([A-ZÁÉÍÓÚÑa-záéíóúñ ]*)', string)[0].strip()
        return [name]

In [3]:
url = 'https://masteres.ugr.es/fisymat/docencia/plan-estudios'
page = urlopen(url)
html_bytes = page.read()
html = html_bytes.decode("utf-8")
html = html#.replace('\t', '').replace('\n', '')
print(html)

<!DOCTYPE html>
<html lang="es" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# " class="no-grid">
  <head>
    <meta charset="utf-8" />
<script>var _paq = _paq || [];(function(){var u=(("https:" == document.location.protocol) ? "https://analiticasweb.ugr.es/" : "http://analiticasweb.ugr.es/");_paq.push(["setSiteId", "400"]);_paq.push(["setTrackerUrl", u+"matomo.php"]);_paq.push(["setDoNotTrack", 1]);if (!window.matomo_search_results_active) {_paq.push(["trackPageView"]);}_paq.push(["setIgnoreClasses", ["no-tracking","colorbox"]]);_paq.push(["enableLinkTracking"]);var d=document,g=d.createElement("script"),s=d.getElementsByTagName("script")[0];g.type="text/ja

# Subject names scraping

In [4]:
subjects_pattern = '''<td>([A-Za-zÁÉÍÓÚáéíóúÑñ\n ]*)</td>'''
[x.replace('\n', '').strip() for x in re.findall(subjects_pattern, html)]

['Astrofísica y Cosmología',
 'Comunicación de la Astrofísica',
 'Física de Galaxias',
 'Física Estelar',
 'Radioastronomía',
 'Técnicas Observacionales en Astrofísica',
 'Análisis Numérico de Edp y Aproximación',
 'Bioestadística y Bioinformática',
 'Física de Redes Complejas y Aplicaciones Interdisciplinares',
 'Modelos Matemáticos en Ecología',
 'Seminario de Invitados Biomat y de Problemas Industriales en Biotecnología',
 'Análisis No Lineal y Ecuaciones Diferenciales',
 'Ecuaciones en Derivadas Parciales Dispersivas No Lineales',
 'Edp de Transporte en Teoría Cinética y Mecánica de Fluidos',
 'Métodos Avanzados de Análisis Funcional y Análisis de Fourier',
 'Principios de Geometría y Aplicaciones en Física',
 'Problemas Variacionales Geométricos',
 'Sistemas Dinámicos y Mecánica',
 'Topología Algebraica y Aplicaciones',
 'Desarrollos Actuales en Física Teórica y Matemáticas y su Fenomenología',
 'Fundamentos Geométricos de la Relatividad General y Gravitación',
 'Introducción a la

# Professors and e-mails scraped

In [5]:
elements = re.findall('<li>([a-zA-ZÁÉÍÓÚáéíóúñ <>/:/\.\-\="\n]*)</li>', html)
elements = [x.strip().replace('\n', '') for x in elements]
elements = [re.split('</li> *<li>', x) for x in elements]
for i in range(len(elements)):
    elements.extend(elements[0])
    elements.remove(elements[0])
elements = [x.strip().replace('  ', ' ') for x in elements]
elements = elements[3:]
elements = [divide(x) for x in elements]
names = pd.DataFrame([x[0] for x in elements], columns=['Name'])
emails = []
for i in range(len(elements)):
    try:
        emails.append(elements[i][1])
    except:
        emails.append(np.nan)
emails = pd.DataFrame(emails, columns=['E-Mail'])
data = pd.concat([names, emails], axis=1)
data

,Name,E-Mail
0,Mónica Relaño Pastor,https://www.ugr.es/personal/monica-relano-pastor
1,Ute Lisenfeld,https://www.ugr.es/personal/ute-lisenfeld
2,Jorge Jiménez Vicente,https://www.ugr.es/personal/jorge-jimenez-vicente
3,Tomás Ruiz Lara,https://www.ugr.es/personal/tomas-ruiz-lara
4,Almudena Zurita Muñoz,https://www.ugr.es/personal/almudena-zurita-munoz
5,María Isabel Pérez Martín,https://www.ugr.es/personal/maria-isabel-perez...
6,Antonio García Hernández,https://www.ugr.es/personal/antonio-garcia-her...
7,Juan Carlos Suárez Yanes,https://www.ugr.es/personal/juan-carlos-suarez...
8,Angela Gardini,NaN
9,Daniel Espada Fernández,https://www.ugr.es/personal/daniel-espada-fern...


# Complete transaction

In [6]:
pattern = '''<td>([A-Za-zÁÉÍÓÚáéíóúÑñ\n \.,]*)</td>[0-9a-zA-ZáéíóúÁÉÍÓÚÑñ<>/"=_\- ]*<li>([a-zA-ZÁÉÍÓÚáéíóúñ <>/:/\.\-\="\n]*)</li>'''
result = re.findall(pattern, html.replace('\n', ''))
elements = []
for i in range(len(result)):
    elements.append(list(result[i]))
del result
for i in range(len(elements)):
    for j in range(len(elements[i])):
        elements[i][j] = elements[i][j].strip()
for i in range(len(elements)):
    elements[i][1] = re.split('</li> *<li>', elements[i][1])
for i in range(len(elements)):
    appender = []
    for j in range(len(elements[i][1])):
        appender.append(divide(elements[i][1][j]))
    elements[i][1] = appender
elements

[['Astrofísica y Cosmología',
  [['Mónica Relaño Pastor',
    'https://www.ugr.es/personal/monica-relano-pastor'],
   ['Ute Lisenfeld', 'https://www.ugr.es/personal/ute-lisenfeld']]],
 ['Comunicación de la Astrofísica',
  [['Jorge Jiménez Vicente',
    'https://www.ugr.es/personal/jorge-jimenez-vicente'],
   ['Tomás Ruiz Lara', 'https://www.ugr.es/personal/tomas-ruiz-lara']]],
 ['Física de Galaxias',
  [['Almudena Zurita Muñoz',
    'https://www.ugr.es/personal/almudena-zurita-munoz'],
   ['María Isabel Pérez Martín',
    'https://www.ugr.es/personal/maria-isabel-perez-martin']]],
 ['Física Estelar',
  [['Antonio García Hernández',
    'https://www.ugr.es/personal/antonio-garcia-hernandez'],
   ['Juan Carlos Suárez Yanes',
    'https://www.ugr.es/personal/juan-carlos-suarez-yanes']]],
 ['Radioastronomía',
  [['Daniel Espada Fernández',
    'https://www.ugr.es/personal/daniel-espada-fernandez']]],
 ['Técnicas Observacionales en Astrofísica',
  [['Laura Sánchez Menguiano',
    'https://w

In [7]:
data = pd.DataFrame(columns=['Subject', 'Professor Name', 'Professor E-Mail'])
k = 0
for i in range(len(elements)):
    subject = elements[i][0]
    for j in range(len(elements[i][1])):
        try:
            email = elements[i][1][j][1]
        except:
            email = np.nan
        data.loc[k] = [subject, elements[i][1][j][0], email]
        k += 1
data

,Subject,Professor Name,Professor E-Mail
0,Astrofísica y Cosmología,Mónica Relaño Pastor,https://www.ugr.es/personal/monica-relano-pastor
1,Astrofísica y Cosmología,Ute Lisenfeld,https://www.ugr.es/personal/ute-lisenfeld
2,Comunicación de la Astrofísica,Jorge Jiménez Vicente,https://www.ugr.es/personal/jorge-jimenez-vicente
3,Comunicación de la Astrofísica,Tomás Ruiz Lara,https://www.ugr.es/personal/tomas-ruiz-lara
4,Física de Galaxias,Almudena Zurita Muñoz,https://www.ugr.es/personal/almudena-zurita-munoz
5,Física de Galaxias,María Isabel Pérez Martín,https://www.ugr.es/personal/maria-isabel-perez...
6,Física Estelar,Antonio García Hernández,https://www.ugr.es/personal/antonio-garcia-her...
7,Física Estelar,Juan Carlos Suárez Yanes,https://www.ugr.es/personal/juan-carlos-suarez...
8,Radioastronomía,Daniel Espada Fernández,https://www.ugr.es/personal/daniel-espada-fern...
9,Técnicas Observacionales en Astrofísica,Laura Sánchez Menguiano,https://www.ugr.es/personal/laura-sanchez-meng...


## Save to database and read

In [8]:
data.to_sql('subjects_data', con=sqlite3.connect('data.db'))

52

In [9]:
con = sqlite3.connect('data.db')
pd.read_sql('SELECT * FROM subjects_data', con=con)

,index,Subject,Professor Name,Professor E-Mail
0,0,Astrofísica y Cosmología,Mónica Relaño Pastor,https://www.ugr.es/personal/monica-relano-pastor
1,1,Astrofísica y Cosmología,Ute Lisenfeld,https://www.ugr.es/personal/ute-lisenfeld
2,2,Comunicación de la Astrofísica,Jorge Jiménez Vicente,https://www.ugr.es/personal/jorge-jimenez-vicente
3,3,Comunicación de la Astrofísica,Tomás Ruiz Lara,https://www.ugr.es/personal/tomas-ruiz-lara
4,4,Física de Galaxias,Almudena Zurita Muñoz,https://www.ugr.es/personal/almudena-zurita-munoz
5,5,Física de Galaxias,María Isabel Pérez Martín,https://www.ugr.es/personal/maria-isabel-perez...
6,6,Física Estelar,Antonio García Hernández,https://www.ugr.es/personal/antonio-garcia-her...
7,7,Física Estelar,Juan Carlos Suárez Yanes,https://www.ugr.es/personal/juan-carlos-suarez...
8,8,Radioastronomía,Daniel Espada Fernández,https://www.ugr.es/personal/daniel-espada-fern...
9,9,Técnicas Observacionales en Astrofísica,Laura Sánchez Menguiano,https://www.ugr.es/personal/laura-sanchez-meng...
